In [1]:
from transformer import LlamaLMHeadModel, Config
import torch
import torch.nn.functional as F
import torch.nn

config = Config()
model = LlamaLMHeadModel(config)
model.load_state_dict(torch.load("nanollama_weights.pth"))
model = model.to(config.device)

C:\Users\HP\AppData\Local\Temp\ipykernel_26652\166369440.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("nanollama_weights.pth"))


In [2]:
import torch

def generate_autoregressive(
    model,
    tokenizer,
    prompt: str,
    max_new_tokens: int = 50,
    temperature: float = 1.0,
    top_k: int = 50,
    eos_token_id: int = 3,
    device: str = "cuda" if torch.cuda.is_available() else "cpu"
):
    model.eval()
    input_ids = tokenizer.encode(prompt, return_tensor=True).to(device)
    
    with torch.no_grad():
        for _ in range(max_new_tokens):
            outputs = model(input_ids)  # logits: [1, seq_len, vocab_size]
            logits = outputs[:, -1, :] / temperature  # take last token's logits
            if top_k is not None:
                top_k_logits, top_k_indices = torch.topk(logits, top_k)
                probs = torch.nn.functional.softmax(top_k_logits, dim=-1)
                next_token = top_k_indices[0, torch.multinomial(probs, 1)]
            else:
                probs = torch.nn.functional.softmax(logits, dim=-1)
                next_token = torch.multinomial(probs, 1)

            # Squeeze the next_token and concatenate with input_ids
            next_token = next_token.squeeze(1)  # Remove extra dimension
            input_ids = torch.cat([input_ids, next_token.unsqueeze(1)], dim=1)  # Concatenate

            if next_token.item() == eos_token_id:
                break

    return tokenizer.decode(input_ids[0].tolist())


In [3]:
from tokenizer import NanoLlamaTokenizer

tokenizer = NanoLlamaTokenizer("nanollama_tokenizer.model")

generated = generate_autoregressive(
    model=model,
    tokenizer=tokenizer,
    prompt="The future of artificial intelligence is",
    max_new_tokens=100,
    top_k=40
)

print(generated)


<s> The future of artificial intelligence is</s> to the largest in the Spanish Empire of life. Sorand of the the Soviet Union of the Soviet Union were also: for example, such as the end of Sté-Da-y-stor, his reigned a small-t-fr-being, but there were also influenced by the first used to have a similar number of the French Revolution in his army, French-parting to, for his brother of the US and for his army; however, who


In [5]:
generated = generate_autoregressive(
    model=model,
    tokenizer=tokenizer,
    prompt="Once upon a time ",
    max_new_tokens=100,
    top_k=40
)

print(generated)

<s> Once upon a time</s>. It has been an important part of the to direct, but also most notably in the same for most prominent of the city in their own very popular in the French Revolution, the early in the early 20th century BC, the the term is believed to holder a small parts of the development of the social factors of the "ID" to be many of the the Sed of the largest" of the American state in the 21st century BC" of the social systems was in a high
